In [3]:
!pip install anomalib

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.8/851.8 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.8/241.8 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.0/846.0 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.5/760.5 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 43.1 MB/s eta 0:00:00
  Created wheel for freia: filename=FrEIA-0.2-py3-none-any.whl size=42763 sha256=0fbcf9f5795c44b8a7ab2e10ea523439573ec88a9b68ed326a86c4da2c1b5f65
  Stored in di

In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import csv
import os
from datetime import datetime
from pathlib import Path
import torch
import cv2
from torchmetrics.classification import BinaryConfusionMatrix
from anomalib.models import Padim, Cfa, Patchcore, Fastflow
from anomalib.models.image.cfa.torch_model import CfaModel
from anomalib.engine import Engine
from anomalib.data import Folder
from torchmetrics.classification import (
    BinaryAccuracy,
    BinaryPrecision,
    BinaryRecall,
    BinaryConfusionMatrix
)

MODEL_NAME = "Padim"
BACKBONE = "resnet18"
LAYERS = ["layer1", "layer2", "layer3", "layer4"]
EPOCHS = 25
CSV_FILENAME = "/content/drive/MyDrive/Hacknation/experiment_results_color_augmanted.csv"
OUTPUT_DIR = "/content/drive/MyDrive/Hacknation/visualizations"


In [8]:
def save_results_to_csv(test_results):
    """
    Zapisuje wyniki, architekturę i parametry do pliku CSV.
    """
    metrics = test_results[0]

    auroc = metrics.get('image_AUROC', 0.0)
    f1_score = metrics.get('image_F1Score', 0.0)



    row = [
        datetime.now().strftime("%Y-%m-%d %H:%M:%S"), # Data i czas
        MODEL_NAME,                                   # Nazwa modelu
        BACKBONE,                                     # Architektura (Backbone)
        str(LAYERS),                                  # Użyte warstwy
        len(LAYERS),                                  # Ilość warstw
        EPOCHS,                                       #EPOCHS
        round(auroc, 4),                              # AUROC (zaokrąglony)
        round(f1_score, 4)                            # F1 Score (zaokrąglony)
    ]

    file_exists = os.path.isfile(CSV_FILENAME)

    try:
        with open(CSV_FILENAME, mode='a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)

            # Jeśli plik jest nowy, dodaj nagłówek
            if not file_exists:
                header = ["Timestamp", "Model", "Backbone", "Layers_List", "Layers_Count", "Epochs", "AUROC", "F1Score"]
                writer.writerow(header)

            # Zapisz dane
            writer.writerow(row)
            print(f"\n[INFO] Wyniki zapisano do pliku: {CSV_FILENAME}")
            print(f"[INFO] AUROC: {round(auroc, 4)} | F1: {round(f1_score, 4)}")

    except Exception as e:
        print(f"[ERROR] Nie udało się zapisać do CSV: {e}")


In [11]:
def train_and_predict():
    # Konfiguracja DataModule
    datamodule = Folder(
        name="hackathon_dataset",
        root='/content/drive/MyDrive/Hacknation',
        normal_dir="train/good",
        abnormal_dir="test/bad",
        train_batch_size=4,
        eval_batch_size=4,
        num_workers=8,
    )
    datamodule.setup()

    #Patchcore
    model = Padim(
        backbone=BACKBONE,
        layers=LAYERS,
        pre_trained=True
    )

    engine = Engine(
        max_epochs=EPOCHS,
        devices=1
    )

    print("Rozpoczynam trening...")
    engine.fit(model=model, datamodule=datamodule)

    print("Rozpoczynam testowanie...")
    test_results = engine.test(model=model, datamodule=datamodule)

    save_results_to_csv(test_results)
    model_path = "/content/drive/MyDrive/Hacknation/saved_models/padim_resnet18_final.ckpt"
    engine.trainer.save_checkpoint(model_path)
    print(f"Model zapisany w: {model_path}")
    return engine, model


In [10]:
train_and_predict()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

INFO:lightning_fabric.utilities.rank_zero:GPU available: False, used: False
INFO:lightning_fabric.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:lightning_fabric.utilities.rank_zero:`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


Rozpoczynam trening...


/usr/local/lib/python3.12/dist-packages/lightning/pytorch/core/optimizer.py:183: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer


...

/usr/local/lib/python3.12/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: The ``compute`` method of
metric F1Score was called before the ``update`` method which may lead to errors, as metric states have not yet been
updated.
  warnings.warn(*args, **kwargs)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │    0.8121621608734131     │
│       image_F1Score       │    0.9736841917037964     │
└───────────────────────────┴───────────────────────────┘

INFO: `weights_only` was not set, defaulting to `False`.
INFO:lightning.pytorch.trainer.connectors.checkpoint_connector:`weights_only` was not set, defaulting to `False`.



[INFO] Wyniki zapisano do pliku: /content/drive/MyDrive/Hacknation/experiment_results_color_augmanted.csv
[INFO] AUROC: 0.8122 | F1: 0.9737
Model zapisany w: /content/drive/MyDrive/Hacknation/saved_models/padim_resnet18_final.ckpt


(<anomalib.engine.engine.Engine at 0x7e40372f5ee0>,
 Padim(
   (pre_processor): PreProcessor(
     (transform): Compose(
           Resize(size=[256, 256], interpolation=InterpolationMode.BILINEAR, antialias=True)
           Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], inplace=False)
     )
     (export_transform): Compose(
           Resize(size=[256, 256], interpolation=InterpolationMode.BILINEAR, antialias=False)
           Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], inplace=False)
     )
   )
   (post_processor): PostProcessor(
     (_image_threshold_metric): F1AdaptiveThreshold()
     (_pixel_threshold_metric): F1AdaptiveThreshold()
     (_image_min_max_metric): MinMax()
     (_pixel_min_max_metric): MinMax()
   )
   (evaluator): Evaluator(
     (val_metrics): ModuleList()
     (test_metrics): ModuleList(
       (0): AUROC()
       (1): F1Score()
       (2): AUROC()
       (3): F1Score()
     )
   )
   (model): PadimModel(
     (feature_ext

In [59]:
!find /content -type d -name "good"

/content/results/Padim/hackathon_dataset/v16/images/good
/content/results/Padim/hackathon_dataset/v1/images/good
/content/results/Padim/hackathon_dataset/v15/images/good
/content/results/Padim/hackathon_dataset/v2/images/good
/content/results/Padim/hackathon_dataset/v17/images/good
/content/results/Padim/hackathon_dataset/v5/images/good
/content/results/Padim/hackathon_dataset/v14/images/good
/content/results/Padim/hackathon_dataset/v4/images/good
/content/results/Padim/hackathon_dataset/v13/images/good
/content/drive/MyDrive/Hacknation/good
/content/drive/.Encrypted/MyDrive/Hacknation/good


In [71]:

!ls drive/MyDrive/Hacknation/visualizations


ls: cannot access 'drive/MyDrive/Hacknation/visualizations': No such file or directory
